In [1]:
from datasets import load_dataset

data_files = {
    "train": "./train/train.csv",
    "validation": "./validation/validation.csv",
    "test": "./test/test.csv"
}

dataset = load_dataset("csv", data_files=data_files)

/home/user/anaconda3/envs/ai/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(dataset["train"][0])

{'text': 'El lloc és acollidor. El tracte, familiar. Els plats són casolans, abundants i de qualitat! Productes de la terra com embutits i altres de la zona. Hi tornarem segur!', 'label': 'positive'}


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("projecte-aina/roberta-base-ca-v2")
model = AutoModelForSequenceClassification.from_pretrained("projecte-aina/roberta-base-ca-v2", num_labels=3)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at projecte-aina/roberta-base-ca-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
label2id = {"negative": 0, "neutral": 1, "positive": 2}

def map_label(example):
    if isinstance(example["label"], str):
        example["label"] = label2id[example["label"].lower()]
    return example

dataset = dataset.map(map_label)

In [ ]:
def preprocess_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 2140/2140 [00:00<00:00, 15576.08 examples/s]


In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets.set_format("torch")

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1_macro = f1_score(labels, predictions, average="macro")
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1_macro": f1_macro}

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

/tmp/ipykernel_33176/2248117808.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.508600,0.533009,0.783645,0.712669
2,0.407300,0.560377,0.792523,0.725820
3,0.318500,0.673324,0.800935,0.734307
4,0.241300,0.757195,0.802336,0.742256


TrainOutput(global_step=8936, training_loss=0.389125874347738, metrics={'train_runtime': 1736.2162, 'train_samples_per_second': 41.17, 'train_steps_per_second': 5.147, 'total_flos': 4701836774799360.0, 'train_loss': 0.389125874347738, 'epoch': 4.0})